# The tale of the snake and the travelling salesman

## Abstract
In this paper we will discuss the mechanics behind the game of Snake and its application in graph theory.

# Objectives
Research and expand upon the following
1. Discuss what is the snake game.
2. How is the snake game related to grid theory?
3. What is a grid?
4. What is the travelling salesman problem?
5. What is a Hamiltonian path and cycle?
6. Implement and discuss an algorithm for generating a Hamiltonian cycle.
7. Implement and discuss an algorithm for the snake to follow the path to the food.

# Bonus objectives (if there's time)
1. Explore the rendering pipeline and how to visualize the snake
2. Research the compute shader step of the pipeline to compute the algorithms in steps 6. and 7.